### Football Data Aggregator
aggregates football results by 2 weeks, month, year

In [120]:
import pandas as pd
import numpy as np

In [121]:
rc = pd.read_csv('results_clean.csv')
rc.drop(["city", "country"],inplace=True, axis=1)
rc.head()

,date,home_team,away_team,home_score,away_score,tournament,neutral,winner,winner_loc
0,2010-03-03,Austria,Denmark,2,1,Friendly,False,Austria,Home
1,2010-03-03,Belgium,Croatia,0,1,Friendly,False,Croatia,Away
2,2010-03-03,France,Spain,0,2,Friendly,False,Spain,Away
3,2010-03-03,Malta,Finland,1,2,Friendly,False,Finland,Away
4,2010-03-03,Poland,Bulgaria,2,0,Friendly,False,Poland,Home


In [122]:
rc["home_win"] = rc["home_team"] == rc["winner"]
rc["away_win"] = rc["away_team"] == rc["winner"]
rc.head()

,date,home_team,away_team,home_score,away_score,tournament,neutral,winner,winner_loc,home_win,away_win
0,2010-03-03,Austria,Denmark,2,1,Friendly,False,Austria,Home,True,False
1,2010-03-03,Belgium,Croatia,0,1,Friendly,False,Croatia,Away,False,True
2,2010-03-03,France,Spain,0,2,Friendly,False,Spain,Away,False,True
3,2010-03-03,Malta,Finland,1,2,Friendly,False,Finland,Away,False,True
4,2010-03-03,Poland,Bulgaria,2,0,Friendly,False,Poland,Home,True,False


In [125]:
# Change this function to change time slice
rc["year_month"] = rc["date"].apply(lambda x: str(x[0:4]) +"-"+ x[5:7])
sw = rc[(rc["year_month"] == "2021-09") & (rc["home_team"]=="Sweden")]
print(sw.head())
rc["games_played"] = 1

home = rc.groupby(["year_month", "home_team"]).sum().add_suffix('').reset_index()
home["away_win"] = 0
home["goal_diff"] = home["home_score"] - home["away_score"]
away = rc.groupby(["year_month", "away_team"]).sum().add_suffix('').reset_index()
away["home_win"] = 0
away["goal_diff"] = away["away_score"] - away["home_score"]

           date home_team away_team  home_score  away_score  \
668  2021-09-02    Sweden     Spain           2           1   

                       tournament  neutral  winner winner_loc  home_win  \
668  FIFA World Cup qualification    False  Sweden       Home      True   

     away_win year_month  games_played  
668     False    2021-09             1  


In [ ]:
comb = home.rename(columns={"home_team":"country"}).append(away.rename(columns={"away_team":"country"}))
comb = comb.groupby(["country", "year_month"]).sum().add_suffix('').reset_index()
comb.drop(["home_score", "away_score"], inplace=True, axis=1)
comb["win_rate"] = (comb["home_win"]+comb["away_win"]) / comb["games_played"]
comb

,country,year_month,neutral,home_win,away_win,games_played,goal_diff,win_rate
0,Austria,2010-03,0,1,0,1,1,1.000000
1,Austria,2010-05,0,0,0,1,-1,0.000000
2,Austria,2010-10,0,0,0,1,0,0.000000
3,Austria,2010-11,0,0,0,1,-1,0.000000
4,Austria,2011-02,0,0,0,1,-2,0.000000
...,...,...,...,...,...,...,...,...
1016,Sweden,2020-11,0,1,0,3,-3,0.333333
1017,Sweden,2021-03,0,1,0,1,1,1.000000
1018,Sweden,2021-05,0,1,0,1,2,1.000000
1019,Sweden,2021-06,2,2,0,3,2,0.666667


In [ ]:
comb.to_csv("results_month.csv", index=False)